In [ ]:
# Скачиваем и объединяем тексты стихов из сборников
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Boris_Pasternak.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Lev_Losev.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Evgeny_Baratynsky.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Boris_Ryzhyi.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Bulat_Okudjava.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Demiyan_Bedny.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Michail_Ancharov.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Nikolay_Gumilev.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Osip_Mandelshtam.txt
!wget https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Vladimir_Nabokov.txt


with open('Bulat_Okudjava.txt', 'r', encoding='utf-8') as f:
    okudjava = f.read()

with open('Demiyan_Bedny.txt', 'r', encoding='utf-8') as f:
    bedny = f.read()

with open('Michail_Ancharov.txt', 'r', encoding='utf-8') as f:
    ancharov = f.read()

with open('Nikolay_Gumilev.txt', 'r', encoding='utf-8') as f:
    gumilev = f.read()

with open('Osip_Mandelshtam.txt', 'r', encoding='utf-8') as f:
    mandelshtam = f.read()

with open('Vladimir_Nabokov.txt', 'r', encoding='utf-8') as f:
    nabokov = f.read()

with open('Boris_Pasternak.txt', 'r', encoding='utf-8') as f:
    pasternak = f.read()

with open('Lev_Losev.txt', 'r', encoding='utf-8') as f:
    losev = f.read()

with open('Evgeny_Baratynsky.txt', 'r', encoding='utf-8') as f:
    bar = f.read()

with open('Boris_Ryzhyi.txt', 'r', encoding='utf-8') as f:
    ryz = f.read()

text = "\\n\\n".join([okudjava, bedny, ancharov, gumilev, mandelshtam, nabokov, pasternak, losev, bar, ryz])  # Объединяем с разделителем

--2025-05-22 21:07:48--  https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Boris_Pasternak.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68528 (67K) [text/plain]
Saving to: ‘Boris_Pasternak.txt’

Boris_Pasternak.txt 100%[===================>]  66.92K  --.-KB/s    in 0.009s  

2025-05-22 21:07:49 (6.90 MB/s) - ‘Boris_Pasternak.txt’ saved [68528/68528]

--2025-05-22 21:07:49--  https://raw.githubusercontent.com/pltnv-a-a/nlp-notes/refs/heads/main/Lev_Losev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
#batch_size = 16 # how many independent sequences will we process in parallel?
#block_size = 32 # what is the maximum context length for predictions?
#max_iters = 5000
eval_interval = 100
#learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
#n_embd = 64
#n_head = 4
#n_layer = 4
#dropout = 0.0
# ------------
batch_size = 32      # Увеличенный размер батча
block_size = 128     # Длина контекста, более подходящая для текста со строфами
max_iters = 30000     # Больше итераций
eval_interval = 200
learning_rate = 3e-4 # Меняем скорость обучения
n_embd = 128         # Увеличенная размерность эмбеддингов
n_head = 8           # Больше голов внимания
n_layer = 6          # Глубже архитектура
dropout = 0.1        # Добавлена регуляризация

torch.manual_seed(1337)

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.rope = RoPE(head_size, block_size)  # RoPE для каждого head
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # Применяем RoPE к query и key
        q = self.rope(q)
        k = self.rope(k)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """Улучшенный FFN с GELU активацией"""
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),  # Более плавная активация
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        return self.weight * self._norm(x)


def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)
    freqs = torch.outer(t, freqs)
    return torch.polar(torch.ones_like(freqs), freqs)  # complex64

class RoPE(nn.Module):
    def __init__(self, dim: int, max_seq_len: int):
        super().__init__()
        self.freqs_cis = precompute_freqs_cis(dim, max_seq_len)

    def forward(self, x: torch.Tensor):
        seq_len = x.size(1)
        freqs_cis = self.freqs_cis[:seq_len].view(1, seq_len, -1).to(x.device)
        x_ = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1, 2))
        x_out = torch.view_as_real(x_ * freqs_cis).flatten(-2)
        return x_out.type_as(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = RMSNorm(n_embd)
        self.ln2 = RMSNorm(n_embd)
        self.alpha = 0.5  # residual scaling factor

    def forward(self, x):
        x = x + self.alpha * self.sa(self.ln1(x))
        x = x + self.alpha * self.ffwd(self.ln2(x))
        return x


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.rope = RoPE(n_embd, block_size)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = RMSNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # улучшенная инициализация
        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        x = self.rope(tok_emb)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=0.8, top_p=0.9):
        """Генерация с температурой и nucleus sampling"""
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature

            # Nucleus sampling
            sorted_logits, sorted_indices = torch.sort(logits, descending=True)
            cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
            logits[indices_to_remove] = -float('Inf')

            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
"""
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx"""

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

1.225755 M parameters
step 0: train loss 5.0726, val loss 5.0755
step 200: train loss 2.7693, val loss 2.8425
step 400: train loss 2.5695, val loss 2.6690
step 600: train loss 2.4546, val loss 2.5735
step 800: train loss 2.3880, val loss 2.5149
step 1000: train loss 2.3365, val loss 2.4635
step 1200: train loss 2.2949, val loss 2.4276
step 1400: train loss 2.2628, val loss 2.4029
step 1600: train loss 2.2259, val loss 2.3696
step 1800: train loss 2.1917, val loss 2.3513
step 2000: train loss 2.1624, val loss 2.3249
step 2200: train loss 2.1370, val loss 2.2947
step 2400: train loss 2.1027, val loss 2.2803
step 2600: train loss 2.0819, val loss 2.2538
step 2800: train loss 2.0607, val loss 2.2323
step 3000: train loss 2.0390, val loss 2.2209
step 3200: train loss 2.0135, val loss 2.1980
step 3400: train loss 1.9928, val loss 2.1882
step 3600: train loss 1.9760, val loss 2.1759
step 3800: train loss 1.9599, val loss 2.1646
step 4000: train loss 1.9322, val loss 2.1496
step 4200: train lo

In [ ]:
context = torch.tensor([encode("осень")], dtype=torch.long, device=device)

print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

осенье -- моей общетил.

Какая пенье больница моих кругой,

не печали в утеполой лет, в руку, скрыться на сторах.

 

А стоялся плотникой землей

себя бы бесседний реской не надежность

по выроскались и полосом и листи,

И светло, над опортите проходят.

 

Закатит звонков и слухи и веком

смеют по бог отдавших мудой одной,

то старин ложит небесет на ворона.

 

Солнце видел не смотрит и верь,

остроить верно сердца собачий,

глубок неба ликолых славоров.

 

Он слушались крошали тебя,

в придумчивай строго ни в огне...

В разнатительниц молодных лет,

в все не жалость не один.

 

На бледный торческий не прекостит –

Поклянет всем другие, всеми,

моя песня, ты в посмотренное из зверя.

 

И в поднимах, как вечером постранничья,

Привостовит на полоть новая лицо.

И не тогда навернет туман.

Вы детствая там, почтет и тепло на глубинки добротью.
 
Ты могуче не стал, я то, скажу, что мне не убержествуять,
Солдейшей ногу, о красной мой сприпней,
И не детский, светлой дорогий будет,
Чтобы

In [ ]:
context = torch.tensor([encode("смерть")], dtype=torch.long, device=device)

print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

смерть,
    Как не за глубинами себя.
    Повесть в комном колено плотница,
        Одиного на мальчиком ты!
Под страстно летела комнать оплоками ветер,
Как замеркались куда из себя гордая,
И не тобою плещалась и ручи толкому виновать,
Как же мирости был за розов презраном глубом.
 
И печалейского порогую и такома,
Но роз в сокровавьев синой,
И в землях мы на колоне другом,
Что на выленнею поют.
 
Все на мировою плиту,
Минус голодные полоды,
И войны городый трудный тебя звезды
Тоже не не победа, в гулов своих.

 

Прокойный рыни молодно всё не против кальцам,

а на слезые губы на страну.

 

Я прохожу я так не стройный труб

Ничего опусть будут просторанский,

Как причувшись, облоколистов,

Как в плату силосьный строну.

 

И будет ты больше пред мой темной повсе,

на глядой сокрое поднимает.

Последням на полном дому и с душой,

Чтобы больше только детская возьм,

От мне отошен смерть, с недострогит.

 

Но со страшник наши огни.

И к передставляется, в подворах.

Твои почум непрослил

In [ ]:
context = torch.tensor([encode("на полустаночке")], dtype=torch.long, device=device)

print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

на полустаночке в сиренье,

     Сумранья в попусту.

 

И в сердце после на воскресенной рощой,

Как небеса и причем в другом прилетел,

когда скажен не много почего пространной лица.

 

Как не за земной человек небесые

как поля бессмерти моей твоею печальной.

О, как мы смерть и пристел маскал на терз.

До крайм топочь с корме и медную темные крылый.

Человечи взглянуть от все дворять в плене, —

по в глубите, чтобы молчатной сеной,

не продлесть следят в ночей барнит

и за волосит не дивной прозрачный,

что я проклятья, годит скажет за белой,

от звенят он подобно, да народ,

старинно и душистый струим плень,

стереть остоит он нашим с радам.


* * *

 

Сборник «Оттор», а не настало –

Да повсем, перевойска, страха.

Как сказки дальнического раскрыть.

Суравшись на стороже, был в кругласть смертный,

и все было в береги наместно

соборник солнце сердце до меня.

 

А тому бы траве и одно на своды.

Перевет, и постоим моим и немного рассе.

Концовым взглянуть сталось западка в глу

In [ ]:
context = torch.tensor([encode("лего")], dtype=torch.long, device=device)

print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

лего,

с протеческой постельной землю,

от войной страшный телеф.

 

Не дышать с дроги попальный,

как по море землю и россился.

Для трепещет он мной тенаря,

Как веки нежный улиц матрарей,

То без руку ночи след грозят,

Подобный снов в моей на свою,

От первой люди, постретит

Морские груди подрого кровь.

 

Что вылетит без полночь

Простих простав в блесне провет,

что он простит дырамим капирым.

 

Напрасным нам мируства вашей столен.

То, что в родной взор странах земной,

Как знающей дари каждый облак,

На стране строй колыбкою друзей.

 

Мой на двух над годы, дума,

Темный от тосках и он до дела,

Только открывает ликом отторгий,

И очень струны и деревья, разве дрожь

На посмотре верят в по тонких отвертил

Белого на серебето вечером,

Так с вороту на скальной и день.

 

И так душе точно нежнее,

А детские жарики, всему открытый,

Держащим только струнит торопе.

 

Ты виносим снами тебя солнце поворно снила,

я скрипет дорогу он не пропит.

Все потомных не привожно роз м

In [ ]:
context = torch.tensor([encode("ты")], dtype=torch.long, device=device)

print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

ты,
И, кто странным волнам переделом,
Прикрасным кардам в плеском продарим.
 
Помню по королам и прекрасно и зловещи.
На лицейских лила своегостной народилась кронит.
 
И оболюбленных камнев и от века
От разговолоси за левочется и дрожавает.
Так в старицу простил с моря спокойный белой,
Старый огоньк правдольный новостью раздора,
Чужали рукованный на медной звуки.
 
Песный луны нарена должной такий земле,
Что заботы рабочей смерти один,
Что все страшно по вырезу запачат вас,
Мы ли там в совест без деле,
И по небе приторного неувидел,
И солнце соловились поролится.
 
Автомоким душом распредины
Наш облаконе отдевши в черной.
В сердце на потом дрожалось,
Чтоб перед пахается нам горячей.
 
   Как полним в виденного на мертвый душат.
 
И не мне смея взорить мира с окниграм,
Велеется по ты страшишь не в обоим.
 
И слезаешь другие окно, полуная беда!
Только же заветала и сказали и они неба,
Солнения, покина смотрел наша с темной,
И полнен божать, как моря, как нас,
И поднимается простил в сно